# Challenge 1

In this challenge you will be working on **Pokemon**. You will answer a series of questions in order to practice dataframe calculation, aggregation, and transformation.

![Pokemon](../images/pokemon.jpg)

Follow the instructions below and enter your code.

#### Import all required libraries.

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.metrics import classification_report, f1_score, cohen_kappa_score
from sklearn.metrics import cohen_kappa_score

In [2]:
%matplotlib inline

#### Import data set.

Read the dataset `pokemon.csv` into a dataframe called `pokemon`.

*Data set attributed to [Alberto Barradas](https://www.kaggle.com/abcsds/pokemon/)*

In [3]:
pokemon = pd.read_csv('Pokemon.csv')

#### Print first 10 rows of `pokemon`.

In [4]:
pokemon.head(10)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False


When you look at a data set, you often wonder what each column means. Some open-source data sets provide descriptions of the data set. In many cases, data descriptions are extremely useful for data analysts to perform work efficiently and successfully.

For the `Pokemon.csv` data set, fortunately, the owner provided descriptions which you can see [here](https://www.kaggle.com/abcsds/pokemon/home). For your convenience, we are including the descriptions below. Read the descriptions and understand what each column means. This knowledge is helpful in your work with the data.

| Column | Description |
| --- | --- |
| # | ID for each pokemon |
| Name | Name of each pokemon |
| Type 1 | Each pokemon has a type, this determines weakness/resistance to attacks |
| Type 2 | Some pokemon are dual type and have 2 |
| Total | A general guide to how strong a pokemon is |
| HP | Hit points, or health, defines how much damage a pokemon can withstand before fainting |
| Attack | The base modifier for normal attacks (eg. Scratch, Punch) |
| Defense | The base damage resistance against normal attacks |
| SP Atk | Special attack, the base modifier for special attacks (e.g. fire blast, bubble beam) |
| SP Def | The base damage resistance against special attacks |
| Speed | Determines which pokemon attacks first each round |
| Generation | Number of generation |
| Legendary | True if Legendary Pokemon False if not |

In [5]:
# Check the index
pokemon.index

RangeIndex(start=0, stop=800, step=1)

#### Obtain the distinct values across `Type 1` and `Type 2`.

Exctract all the values in `Type 1` and `Type 2`. Then create an array containing the distinct values across both fields.

In [6]:
print(pokemon['Type 1'].nunique())
print(pokemon['Type 1'].value_counts())#normalize=True))
pokemon['Type 2'].unique()

18
Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Dragon       32
Ground       32
Ghost        32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64


array(['Poison', nan, 'Flying', 'Dragon', 'Ground', 'Fairy', 'Grass',
       'Fighting', 'Psychic', 'Steel', 'Ice', 'Rock', 'Dark', 'Water',
       'Electric', 'Fire', 'Ghost', 'Bug', 'Normal'], dtype=object)

In [7]:
print(pokemon['Type 2'].nunique())
print(pokemon['Type 2'].value_counts())
pokemon['Type 2'].unique()

18
Flying      97
Ground      35
Poison      34
Psychic     33
Fighting    26
Grass       25
Fairy       23
Steel       22
Dark        20
Dragon      18
Water       14
Ghost       14
Ice         14
Rock        14
Fire        12
Electric     6
Normal       4
Bug          3
Name: Type 2, dtype: int64


array(['Poison', nan, 'Flying', 'Dragon', 'Ground', 'Fairy', 'Grass',
       'Fighting', 'Psychic', 'Steel', 'Ice', 'Rock', 'Dark', 'Water',
       'Electric', 'Fire', 'Ghost', 'Bug', 'Normal'], dtype=object)

In [8]:
types = np.array([pokemon['Type 1'].value_counts().index,pokemon['Type 2'].value_counts().index])
types

array([['Water', 'Normal', 'Grass', 'Bug', 'Psychic', 'Fire', 'Electric',
        'Rock', 'Dragon', 'Ground', 'Ghost', 'Dark', 'Poison', 'Steel',
        'Fighting', 'Ice', 'Fairy', 'Flying'],
       ['Flying', 'Ground', 'Poison', 'Psychic', 'Fighting', 'Grass',
        'Fairy', 'Steel', 'Dark', 'Dragon', 'Water', 'Ghost', 'Ice',
        'Rock', 'Fire', 'Electric', 'Normal', 'Bug']], dtype=object)

In [9]:
# Display an example of the combinations of the two types
pokemon[['Type 1','Type 2']][(pokemon['Type 1'] == 'Electric') & ~(pokemon['Type 2'].isna()) ]

,Type 1,Type 2
88,Electric,Steel
89,Electric,Steel
157,Electric,Flying
196,Electric,Dragon
513,Electric,Steel
531,Electric,Ghost
532,Electric,Fire
533,Electric,Water
534,Electric,Ice
535,Electric,Flying


#### Cleanup `Name` that contain "Mega".

If you have checked out the pokemon names carefully enough, you should have found there are junk texts in the pokemon names which contain "Mega". We want to clean up the pokemon names. For instance, "VenusaurMega Venusaur" should be "Mega Venusaur", and "CharizardMega Charizard X" should be "Mega Charizard X".

In [10]:

# Test code
#pokemon.Name[:10]
#pokemon.Name[3]# .find('Mega')
#pokemon.Name[3][pokemon.Name[3].find('Mega'):]
#pokemon.Name[2].find('Mega') #[pokemon.Name[2].find('Mega'):]



# Answer

pokemon.Name = pokemon.Name.apply(lambda x: x[x.find('Mega'):] if x.find('Mega') != -1 else x)


# test transformed data
pokemon.head(10)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,Mega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,Mega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,Mega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False


In [11]:
# By the way I am checking NaNs
pokemon.isna().sum()

#               0
Name            0
Type 1          0
Type 2        386
Total           0
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

#### Create a new column called `A/D Ratio` whose value equals to `Attack` devided by `Defense`.

For instance, if a pokemon has the Attack score 49 and Defense score 49, the corresponding `A/D Ratio` is 49/49=1.

In [12]:
pokemon['A/D Ratio'] = pokemon.Attack/pokemon.Defense
pokemon['A/D Ratio']

0      1.000000
1      0.984127
2      0.987952
3      0.813008
4      1.209302
         ...   
795    0.666667
796    1.454545
797    1.833333
798    2.666667
799    0.916667
Name: A/D Ratio, Length: 800, dtype: float64

#### Identify the pokemon with the highest `A/D Ratio`.

In [13]:
pokemon.Name[pokemon['A/D Ratio'] == max(pokemon['A/D Ratio'])][429] 

# !!! the filtered slices inherit the index

'DeoxysAttack Forme'

#### Identify the pokemon with the lowest A/D Ratio.

In [14]:
pokemon.Name[pokemon['A/D Ratio'] == min(pokemon['A/D Ratio'])][230]

'Shuckle'

#### Create a new column called `Combo Type` whose value combines `Type 1` and `Type 2`.

Rules:

* If both `Type 1` and `Type 2` have valid values, the `Combo Type` value should contain both values in the form of `<Type 1> <Type 2>`. For example, if `Type 1` value is `Grass` and `Type 2` value is `Poison`, `Combo Type` will be `Grass-Poison`.

* If `Type 1` has valid value but `Type 2` is not, `Combo Type` will be the same as `Type 1`. For example, if `Type 1` is `Fire` whereas `Type 2` is `NaN`, `Combo Type` will be `Fire`.

In [15]:
pokemon['Type 2'][4]

nan

In [16]:
pd.isna(pokemon['Type 2'][4])

True

In [17]:
# after crashing my head against the wall I find out that nan here are float

#for x in pokemon['Type 2']:
#    if not isinstance(x,str):
#        print(x,type(x))

In [18]:
# I prefer the lambda. Neither of them worked because of the strange nan class float

#def joinTypes(x,y):
#    if y != np.nan:
#        return x + '-' + y
#    else:
#        return x 

In [19]:
# pokemon['Combo Type'] = list(map(joinTypes, pokemon['Type 1'], pokemon['Type 2']))


# ??? Why does not work with lambda?
pokemon['Combo Type'] = list(map(lambda x,y: x + '-' + y if isinstance(y,str) else x
                                 , pokemon['Type 1'], pokemon['Type 2']))
# TypeError: can only concatenate str (not "float") to str


pokemon.head(20)

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,A/D Ratio,Combo Type
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,1.000000,Grass-Poison
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,0.984127,Grass-Poison
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,0.987952,Grass-Poison
3,3,Mega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,0.813008,Grass-Poison
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,1.209302,Fire
5,5,Charmeleon,Fire,NaN,405,58,64,58,80,65,80,1,False,1.103448,Fire
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False,1.076923,Fire-Flying
7,6,Mega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False,1.171171,Fire-Dragon
8,6,Mega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False,1.333333,Fire-Flying
9,7,Squirtle,Water,NaN,314,44,48,65,50,64,43,1,False,0.738462,Water


#### Identify the pokemon whose `A/D Ratio` are among the top 5.

In [20]:
pokemon[['Name','A/D Ratio']].sort_values('A/D Ratio',ascending=False)[:5]

,Name,A/D Ratio
429,DeoxysAttack Forme,9.000
347,Carvanha,4.500
19,Mega Beedrill,3.750
453,Cranidos,3.125
348,Sharpedo,3.000


#### For the 5 pokemon printed above, aggregate `Combo Type` and use a list to store the unique values.

Your end product is a list containing the distinct `Combo Type` values of the 5 pokemon with the highest `A/D Ratio`.

In [21]:
pokemon[['Name','Combo Type','A/D Ratio']].sort_values('A/D Ratio',ascending=False)[:5]

,Name,Combo Type,A/D Ratio
429,DeoxysAttack Forme,Psychic,9.000
347,Carvanha,Water-Dark,4.500
19,Mega Beedrill,Bug-Poison,3.750
453,Cranidos,Rock,3.125
348,Sharpedo,Water-Dark,3.000


In [22]:
topCombos = list(pokemon[['Name','Combo Type','A/D Ratio']].sort_values('A/D Ratio',ascending=False)['Combo Type'][:5])
topCombos



['Psychic', 'Water-Dark', 'Bug-Poison', 'Rock', 'Water-Dark']

In [ ]:
# Values are not unique
# Let's try with group by

In [35]:
maxAD = pokemon[['Combo Type','A/D Ratio']].groupby(['Combo Type']).agg(max)
maxAD.sort_values('A/D Ratio',ascending=False)

topCombos = list(maxAD.sort_values('A/D Ratio',ascending=False)[:5].index)
topCombos



['Psychic', 'Water-Dark', 'Bug-Poison', 'Rock', 'Steel-Ghost']

#### For each of the `Combo Type` values obtained from the previous question, calculate the mean scores of all numeric fields across all pokemon.

Your output should look like below:

![Aggregate](../images/aggregated-mean.png)

In [43]:
pokemon[pokemon['Combo Type'].isin(topCombos)].groupby(['Combo Type']).agg(pd.Series.mean) # Dataframe


/Users/albertomengual/opt/anaconda3/lib/python3.9/site-packages/pandas/core/groupby/generic.py:303: FutureWarning: Dropping invalid columns in SeriesGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the aggregating function.
  results[key] = self.aggregate(func)


,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,A/D Ratio
Combo Type,,,,,,,,,,,
Bug-Poison,199.166667,347.916667,53.750000,68.333333,58.083333,42.500000,59.333333,65.916667,2.333333,0.000000,1.315989
Psychic,381.973684,464.552632,72.552632,64.947368,67.236842,98.552632,82.394737,78.868421,3.342105,0.236842,1.164196
Rock,410.111111,409.444444,67.111111,103.333333,107.222222,40.555556,58.333333,32.888889,3.888889,0.111111,1.260091
Steel-Ghost,680.250000,453.250000,56.000000,97.500000,112.500000,70.000000,71.500000,45.750000,6.000000,0.000000,1.216667
Water-Dark,347.666667,493.833333,69.166667,120.000000,65.166667,88.833333,63.500000,87.166667,3.166667,0.000000,2.291949
